# HTMD FFEvaluate - Easy MM force-field evaluation

HTMD provides the FFEvaluate module which allows the easy evaluation of molecular mechanics force-fields. This serves multiple purposes:

 * __Application development__: Many computational biology applications require the evaluation of energies between molecules. This allows faster development of applications based on those energies.
 * __Educational__: Most molecular dynamics codes are either closed source or written in a highly optimized manner making them hard to understand. The FFEvaluate code has been written in python numba which makes it easy to read, improve and extend.
 * __Debugging__: FFEvaluate can provide a detailed breakdown of all energies and visualization of forces which helps debug problems in a MD simulation or parameterization.
 
We would like to acknowledge [parmed](), an awesome python library which FFEvaluate uses to parse forcefield parameters.

## Quick example

In [ ]:
from htmd.ffevaluation.ffevaluate import FFEvaluate, loadParameters
from htmd.molecule.molecule import Molecule

# Load the parameters
prm = loadParameters('./structure.prmtop')  
# Load the topology
mol = Molecule('./structure.prmtop')  
# Create a FFEvaluate object
ffev = FFEvaluate(mol, prm)

# Load some coordinates
mol.read('./structure.pdb')
# Calculate energy and forces
energies, forces, atom_energies = ffev.calculate(mol.coords)  
# Calculate interaction energy between ligand and protein
energies = ffev.calculateEnergies(mol.coords, betweensets=('protein', 'resname BEN'))

## Detailed explanation

In [ ]:
from htmd.ffevaluation.ffevaluate import FFEvaluate, loadParameters
from htmd.molecule.molecule import Molecule

The `loadParameters` function serves as a utility function for loading `AMBER` and `CHARMM` forcefield parameters. It is simply a wrapper for the `parmed` loading functions as it can sometimes get a bit complex to obtain parameters from some files, but if you already know how to use `parmed` you can use it directly to obtain a `ParameterSet` object of your parameter files.

In [ ]:
prm = loadParameters('./structure.prmtop')

Now that we have loaded the parameters, we load the `prmtop` topology in a `Molecule` object to obtain the bond, angle, dihedral, improper and atomtype information. 

In [ ]:
mol = Molecule('./structure.prmtop')

The main class for the calculations is the `FFEvaluate` class. When instantiated, the class constructor converts and stores the parameters in a format friendly to the `numba python` calculations. This is only done once for each system & parameterization combination as it incurrs a small computational overhead. The energy calculations are then done with the class methods `calculate` and `calculateEnergies` which can be evaluated for multiple different configurations of the system without having to incurr the same overhead by re-converting the parameters at every call.

In [ ]:
ffev = FFEvaluate(mol, prm)

Now let's read in some coordinates into our Molecule to evaluate the energies and forces.

In [ ]:
mol.read('./structure.pdb')
energies, forces, atom_energies = ffev.calculate(mol.coords)

As we can see, FFEvaluate returns:

* The energy of the system broken down in bonded, angle, dihedral, improper, VdW and electrostatic terms in a (6, nframes) shaped array
* The forces of all atoms in a (natoms, 3, nframes) shaped array
* The energies of each individual atom in a (natoms, 6, nframes) shaped array. These energies are not physically meaningful as they are approximate atom energies calculated as the sum of all non-bonded energies on that atom plus the fraction of the bonded energy terms (i.e. for bonds we divide equally the bond energy between the two atoms, for angles between all three atoms and for dihedrals/impropers between all 4 atoms). However they can be useful for identifying issues in a parameterization or high-energy areas such as collisions.

As we often want to just obtain energies in a nicely formatted fashion, the class also provides the `calculateEnergies` utility method which performs the same calculations but returns only the energies in a python dictionary.

In [ ]:
ffev.calculateEnergies(mol.coords)

Lastly, we often are interested specifically in (non-bonded) interaction energies between two molecules or generally between two sets of atoms. FFEvaluate allows us to calculate these energies using the `betweensets` argument. This argument accepts a list or tuple of two `atomselect` strings which define the two atom sets.

In [ ]:
energies = ffev.calculateEnergies(mol.coords, betweensets=('protein', 'resname BEN'))

## Disclaimers

While FFEvaluate provides an easy and fast way of obtaining MM energies and forces, the user should be aware of some pitfalls.

 * The code is still in beta. There are some less-user or older force-field features which are not yet supported (i.e. Urey-Bradley types, explicit AMBER 1-4 exclusions)
 * It will never be as fast as professional MD codes. The purpose of `FFEvaluate` is to make an easily usable and understandable MM evaluation code. Therefore on systems with large amount of non-bonded interations such as solvated systems with many waters the code __will__ take a long time to evaluate the energies and forces.
 * Speed improvements such as cell-lists for non-bonded interations and particle-mesh Ewald are not implemented (but could be in the future). FFEvaluate will calculate all non-bonded interactions explicitly.
 * Implicit solvent calculations are still missing but might be added in the future.
 
If you are interested in contributing code to the project, or want to report a bug, feel free to chat us up on the [HTMD github issues](https://github.com/acellera/htmd/issues/)!